# This Program is developed by Soheil Hekmat For Q3 part a and b

Importing Libraries

In [13]:
from gym import Env
from gym.spaces import Discrete
import numpy as np
import random

Making The Environment Class

In [163]:
class SoheilEnv(Env):
    def __init__(self):
        # Actions we can take, we can have 4 actions in diameter
        self.action_space = Discrete(4)
        # all states array, not that we have states 1 to 99 but in array it's 0 to 98 so we should use +1 in every use
        self.observation_space = Discrete(99)
        # Set start states s1 and s2
        st = [81,85]
        self.state = random.choice(st)
        # Set maximum step
        self.max_step = 200
        self.grid = [['.' for _ in range(11)] for _ in range(9)]

    def step(self, action):
        # Apply action
        ac =[-12, -10, 10, 12]
        # 0 == -12
        # 1 == -10
        # 2 == 10
        # 3 == 12
        self.state += ac[action]
        # Reduce maximum step by 1 time
        self.max_step -= 1

        # Set a default value for reward in case it's not set in your environment
        reward = 0
        # Calculate reward
        white = [15,16,17,18,19,20,21,24,25,26,27,28,31,32,35,36,37,42,43,47,48,49,50,51,52,53,54,57,58,62,63,68,69,70,71,72,73,74,75,76,79,80,81,82,84,85,86,87]
        gray = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,22,23,30,33,34,38,39,40,41,44,45,46,55,56,59,60,61,64,65,66,67,77,78,83,88,89,90,91,92,93,94,95,96,97,98,99]
        green = 29
        if self.state in white:
            reward =-1
        if self.state in gray:
            reward = -5
        if self.state == green:
            reward = 8

        # Check if path finding is done
        if self.state in gray or self.state == green:
            done = True
        else:
            done = False

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        for row in range(9):
          for col in range(11):
            if row == 0 or row == 8 or col == 0 or col ==10:
              self.grid[row][col] = 'X'
            if (row,col) == (1,1) or (row,col) == (1,2) or (row,col) == (2,7) or (row,col) == (3,4) or (row,col) == (3,5) or (row,col) == (3,6) or (row,col) == (3,7):
              self.grid[row][col] = 'X'
            if (row,col) == (4,1) or (row,col) == (5,3) or (row,col) == (5,4) or (row,col) == (5,5) or (row,col) == (5,8) or (row,col) == (5,9) or (row,col) == (7,5):
              self.grid[row][col] = 'X'
        for i in range(9):
          for j in range(11):
            if (i,j) == (3-1,7-1):
              print('G', end=' ')
            if (i,j) == (8-1,4-1):
              print('S', end=' ')
            if (i,j) == (8-1,8-1):
              print('S', end=' ')
            if (i,j) != (8-1,8-1) and (i,j) != (8-1,4-1) and (i,j) != (3-1,7-1):
              print(self.grid[i][j], end=' ')
          print()
        print()
    def reset(self):
        # Reset robot's position to s1 or s2
        st = [81,85]
        self.state = random.choice(st)
        # Reset max step
        self.max_step = 200
        # Initialize the grid with special characters
        # for i in range(9):
        #     for j in range(11):
        #         for cell, symbol in self.special_cells:
        #             if (i, j) == cell:
        #                 self.grid[i][j] = symbol

        # Return state adjusted to the range 1 to 99 (descrete(99) is 0 to 98)
        return self.state


Environment calling

In [164]:
env = SoheilEnv()

In [59]:
env.action_space.sample()
#env.observation_space.sample() + 1

3

Testing some actions and rewards

In [167]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} FinalState:{}'.format(episode, score, n_state))

Episode:1 Score:-7 FinalState:93
Episode:2 Score:-8 FinalState:59
Episode:3 Score:-5 FinalState:95
Episode:4 Score:-7 FinalState:93
Episode:5 Score:-6 FinalState:65
Episode:6 Score:-5 FinalState:95
Episode:7 Score:-5 FinalState:95
Episode:8 Score:-5 FinalState:91
Episode:9 Score:-10 FinalState:65
Episode:10 Score:-5 FinalState:91


# Visualization

In [128]:
env.render()

X X X X X X X X X X X 
X X X . . . . . . . X 
X . . . . . G X . . X 
X . . . X X X X . . X 
X X . . . . . . . . X 
X . . X X X . . X X X 
X . . . . . . . . . X 
X . . S . X . S . . X 
X X X X X X X X X X X 



In [ ]:
env.observation_space.n

In [165]:
# defining env and num actions and states
num_states = env.observation_space.n
num_actions = env.action_space.n
# Initialize the transition probability matrix P
P = np.zeros((num_states, num_actions, num_states))

# Fill in the transition probability matrix P based on your environment's dynamics

# Initialize the state value function V, state-action value function Q, and policy
V = np.zeros(num_states)
Q = np.zeros((num_states, num_actions))
policy = np.zeros(num_states+1, dtype=int)

# Policy Iteration
def policy_iteration():
    max_steps = 1000
    gamma = 1

    for _ in range(max_steps):
        state = env.reset()
        step = 0
        terminated = False
        # Policy Evaluation
        # for s in range(num_states):
        s = 0
        while not terminated:
            action = policy[s]
            next_state, reward, terminated, _ = env.step(action)
            P = np.zeros((num_states, num_actions, num_states))
            for s in range(num_states):
              for a in range(num_actions):
                if a == 0:  # action -12
                    P[s, a, max(s - 12, 0)] = 1
                elif a == 1:  # action -10
                    P[s, a, max(s - 10, 0)] = 1
                elif a == 2:  # action +10
                    P[s, a, min(s + 10, 98)] = 1
                elif a == 3:  # action +12
                    P[s, a, min(s + 12, 98)] = 1
            V[s] = reward + gamma * np.sum(P[s, action, :] * V)
            Q[s, action] = reward + gamma * np.sum(P[s, action, :] * V)
            if terminated:
                break
            print(Q[s, action])
            s +=1
        # Policy Improvement
        policy_stable = True
        for s in range(num_states):
            old_action = policy[s]
            policy[s] = np.argmax(Q[s, :])
            if old_action != policy[s]:
                policy_stable = False

        if policy_stable:
            break

    return policy, Q

# Value Iteration
def value_iteration():
    max_steps = 1000
    gamma = 1

    for _ in range(max_steps):
        for s in range(num_states):
            action_values = np.sum(P[s, :, :] * (1 + gamma * V), axis=1)
            policy[s] = np.argmax(action_values)
            Q[s, :] = action_values
            V[s] = np.max(action_values)

    return policy, Q

# Run Policy Iteration
policy_pi, Q_pi = policy_iteration()

# Run Value Iteration
policy_vi, Q_vi = value_iteration()

# Print results
print("Policy Iteration - Optimal Policy:")
print(policy_pi)
print("Policy Iteration - Optimal State-Action Values:")
print(Q_pi)

print("\nValue Iteration - Optimal Policy:")
print(policy_vi)
print("Value Iteration - Optimal State-Action Values:")
print(Q_vi)


-1.0
-1.0
-1.0
Policy Iteration - Optimal Policy:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Policy Iteration - Optimal State-Action Values:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 

# Path

policy iteration

In [ ]:
# Print sequence of actions that has taken to reach the goal
Actions =  {0: 'Up Right',
            1: 'Up Forward',
            2: 'Down Left',
            3: 'Down Right'}
Qtable_selected = Q_pi
action_sequence=[]
for j in range(Qtable_selected.shape[0]):
    action_sequence.append(Actions[np.argmax(np.array(Qtable_selected)[j, :])])
print(' --> '.join(action_sequence))

value iteration

In [ ]:
Qtable_selected = Q_vi
action_sequence=[]
for j in range(Qtable_selected.shape[0]):
    action_sequence.append(Actions[np.argmax(np.array(Qtable_selected)[j, :])])
print(' --> '.join(action_sequence))